In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
from pylab import *
from skimage import measure,draw,data
import json


In [2]:
def close_contour(contour):
    if not np.array_equal(contour[0], contour[-1]):
        contour = np.vstack((contour, contour[0]))
    return contour

In [3]:
def create_sub_masks(mask_image):#将三个子mask分离出来
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]
            
            # Check to see if we've created a sub-mask...
            pixel_str = str(pixel)
            sub_mask = sub_masks.get(pixel_str)
            if sub_mask is None:
                # Create a sub-mask (one bit per pixel) and add to the dictionary
                # Note: we add 1 pixel of padding in each direction
                # because the contours module doesn't handle cases
                # where pixels bleed to the edge of the image
                sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

            # Set the pixel value to 1 (default is 0), accounting for padding
            sub_masks[pixel_str].putpixel((x+1, y+1), 1)
            
    maps = {"(0, 0, 0)":"background","(0, 128, 0)":"pipe","(128, 0, 0)":"valve"}
    for v in list(sub_masks.keys()):
        sub_masks[maps[v]] = sub_masks.pop(v)

    return sub_masks

In [4]:
#binary_mask_to_polygon
#mask2polygon
#将二值mask转换为segmentation
#create_sub_masks
#将三通道原始mask图像转换为n个子mask


In [5]:
def binary_mask_to_polygon(binary_mask, tolerance=0):
    """Converts a binary mask to COCO polygon representation
    Args:
        binary_mask: a 2D binary numpy array where '1's represent the object
        tolerance: Maximum distance from original points of polygon to approximated
            polygonal chain. If tolerance is 0, the original coordinate array is returned.
    """
    polygons = []
    # pad mask to close contours of shapes which start and end at an edge
    padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
    contours = measure.find_contours(padded_binary_mask, 0.5)
    
    for c in contours:
        c = np.array(c)
    
    #contours = np.subtract(contours, 1)
    for contour in contours:
        contour = np.subtract(contour,1)
        contour = close_contour(contour)
        contour = measure.approximate_polygon(contour, tolerance)
        if len(contour) < 3:
            continue
        contour = np.flip(contour, axis=1)
        segmentation = contour.ravel().tolist()
        # after padding and subtracting 1 we may get -0.5 points in our segmentation
        segmentation = [0 if i < 0 else i for i in segmentation]
        polygons.append(segmentation)

    return polygons

In [6]:
def mask2polygon(mask):
    contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    # mask_new, contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    segmentation = []
    for contour in contours:
        contour_list = contour.flatten().tolist()
        if len(contour_list) > 4:# and cv2.contourArea(contour)>10000
            segmentation.append(contour_list)
    return segmentation

In [7]:
def bounding_box(img):
    #计算包围盒
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]

    return rmin, rmax, cmin, cmax
def countArea(box):
    #计算area
    return abs((box[1]-box[0])*(box[3]-box[2]))

In [8]:
#coco数据格式
s = {
    "info": {},
    "categories": [
        {
            "id": 0,
            "name": "background",
            "supercategory": 0,
            "metadata": {},
            "color": "#000000"
        },
        {
            "id": 1,
            "name": "pipe",
            "supercategory": 0,
            "metadata": {},
            "color": "#008000"
        },
        {
            "id": 2,
            "name": "valve",
            "supercategory": 0,
            "metadata": {},
            "color": "#800000"
        }
    ],
    "images": [
    ],
    "annotations": [
    ]
}
        

In [4]:
import os

In [2]:
maskdir = "masks/"
color = {"background":"#000000","pipe":"#008000","valve":"#800000"}
catid = {"background":0,"pipe":1,"valve":2}

In [5]:
masks = os.listdir(maskdir)

In [12]:
segid = 0 #segmentation的id
for index in range(len(masks)):
    #打开文件，转换为RGB格式
    img = Image.open(maskdir+masks[index]).convert("RGB")
    w,h = img.size #宽和高
    filename = masks[index]
    path = maskdir + filename
    imgs = {"id":index,"width":w,"height":h,"file_name":filename,"path":path,"license":0,"fickr_url":0,"coco_url":0,"data_captured":0,"metadata":{}}
    s["images"].append(imgs)
    #将mask分离出子mask
    res  = create_sub_masks(img)
    print("progress:"+str(index))
    #对于每一个子mask，转换成segementation
    for m in res.keys():
        submask = np.array(res[m])
        segment = mask2polygon(submask)
        bbox = list(bounding_box(submask))
        area = countArea(bbox)
        anno = {"id":segid,"image_id":index,"category_id":catid[m],"area":area,"segmentation":segment,"bbox":bbox,"metadata": {},"color":color[m]}
        segid += 1
        #将segmentation放入字典
        s["annotations"].append(anno)
      
    

progress:0
progress:1
progress:2
progress:3
progress:4
progress:5
progress:6
progress:7
progress:8
progress:9
progress:10
progress:11
progress:12
progress:13
progress:14
progress:15
progress:16
progress:17
progress:18
progress:19
progress:20
progress:21
progress:22
progress:23
progress:24
progress:25
progress:26
progress:27
progress:28
progress:29
progress:30
progress:31
progress:32
progress:33
progress:34
progress:35
progress:36
progress:37
progress:38
progress:39
progress:40
progress:41
progress:42
progress:43
progress:44
progress:45
progress:46
progress:47
progress:48
progress:49
progress:50
progress:51
progress:52
progress:53
progress:54
progress:55
progress:56
progress:57
progress:58


In [13]:
file_handle=open('a.json',mode='w')
file_handle.write(str(s))
file_handle.close()

In [14]:

import io
def alter(file,old_str,new_str):
    """
    替换文件中的字符串
    :param file:文件名
    :param old_str:就字符串
    :param new_str:新字符串
    :return:
    """
    file_data = ""
    with io.open(file, "r", encoding="utf-8") as f:
        for line in f:
            if old_str in line:
                line = line.replace(old_str,new_str)
            file_data += line
    with io.open(file,"w",encoding="utf-8") as f:
        f.write(file_data)
        
alter("a.json","\'","\"")
alter("a.json","png","jpg")

In [18]:
# /JPEGImages/2007_000032.jpg /SegmentationClassAug/2007_000032.png
s = " "
file_handle=open('train.txt',mode='w')
for mask in masks:
    s = "/JPEGImages/"+mask[:-4]+".jpg"+ " /SegmentationClass/"+mask+"\n"
    file_handle.write(s)

In [19]:

file_handle.close()